<a href="https://colab.research.google.com/github/dangni02/dacos_rec/blob/main/collaborative_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 구글 드라이브 마운트 (코랩에서 실행)
from google.colab import drive
drive.mount('/content/drive')

# 데이터 로드
restaurant_data = pd.read_csv("/content/drive/MyDrive/content/df_meal.csv", encoding='euc-kr')
cafe_data = pd.read_csv("/content/drive/MyDrive/content/df_cafe.csv", encoding='euc-kr')

# 태그 옵션 정의 (술집 관련 태그를 식당에 통합)
tag_options = {
    '식당': {
        '맛': ['음식이 맛있어요', '빵이 맛있어요'],
        '특별한 메뉴': ['특별한 메뉴가 있어요'],
        '메뉴 다양성': ['메뉴 구성이 알차요'],
        '신선도': ['재료가 신선해요', '잡내가 적어요', '고기 질이 좋아요'],
        '가성비': ['가성비가 좋아요', '양이 많아요'],
        '서비스': ['친절해요', '반찬이 잘 나와요', '음식이 빨리 나와요', '포장이 깔끔해요'],
        '청결': ['매장이 청결해요', '화장실이 깨끗해요'],
        '혼밥' : ['혼밥하기 좋아요'],
        # 술집 관련 태그를 식당에 통합
        '다양한 술': ['술이 다양해요'],
        '분위기': ['음악이 좋아요', '아늑해요', '인테리어가 멋져요'],
        '대화하기 좋음': ['대화하기 좋아요']
    },
    '카페': {
        '커피': ['커피가 맛있어요'],
        '디저트': ['디저트가 맛있어요'],
        '청결': ['매장이 청결해요', '화장실이 깨끗해요'],
        '서비스': ['친절해요', '포장이 깔끔해요'],
        '분위기': ['아늑해요', '인테리어가 멋져요'],
        '공부하기 좋음': ['집중하기 좋아요', '좌석이 편해요', '매장이 넓어요']
    }
}

# 식당만 고정된 음식 종류
category_options = {
    '식당': ['한식', '중식', '일식', '양식', '육류', '아시안음식', '샐러드']
}

# 사용자 입력 처리 함수 (다중 선택)
def get_user_selection_multi(prompt, choices):
    """
    사용자로부터 다중 선택을 받는 함수
    - prompt: 사용자에게 보여줄 메시지
    - choices: 선택지 리스트

    Returns:
    - selected_choices: 사용자가 선택한 항목들의 리스트
    """
    while True:
        print(prompt)
        for i, choice in enumerate(choices, 1):
            print(f"{i}. {choice}")
        try:
            selections = input("선택 번호를 입력하세요 (쉼표로 구분): ")
            selected_indices = [int(x.strip()) - 1 for x in selections.split(',')]
            if all(0 <= idx < len(choices) for idx in selected_indices):
                selected_choices = [choices[idx] for idx in selected_indices]
                return selected_choices
            else:
                print(f"잘못된 입력입니다. 1부터 {len(choices)} 사이의 숫자를 입력하세요.")
        except ValueError:
            print("숫자와 쉼표만 입력하세요.")

# 새로운 랭킹 함수 (다중 태그 지원)
def rank_stores_custom_multi(data, selected_tags, tag_options):
    """
    선택한 태그들을 기준으로 가게들을 랭킹하는 함수
    - 전체적인 태그 리뷰 수: 50%
    - 절대적 해당 키워드 태그 리뷰 수: 30%
    - 상대적 해당 키워드 태그 리뷰 수: 20%

    Parameters:
    - data: 필터링된 데이터프레임
    - selected_tags: 사용자가 선택한 태그 리스트
    - tag_options: 선택한 카테고리의 태그 옵션 딕셔너리

    Returns:
    - ranked_data: 랭킹된 데이터프레임 (최소 7개 이상)
    """

    # 선택한 태그에 해당하는 리뷰 컬럼들 가져오기
    selected_phrases = []
    for tag in selected_tags:
        selected_phrases.extend(tag_options[tag])

    # 절대적 해당 키워드 태그 리뷰 수 계산
    data['Selected_Tag_Absolute'] = data[selected_phrases].sum(axis=1)

    # 전체적인 태그 리뷰 수 (이미 '태그 총합' 컬럼에 있음)
    data['Overall_Tag_Count'] = data['태그 총합']

    # 상대적 해당 키워드 태그 리뷰 수 계산
    data['Selected_Tag_Relative'] = data['Selected_Tag_Absolute'] / data['Overall_Tag_Count']

    # 각 기준을 정규화 (0 ~ 1 범위)
    data['Overall_Tag_Normalized'] = data['Overall_Tag_Count'] / data['Overall_Tag_Count'].max()
    data['Selected_Tag_Absolute_Normalized'] = data['Selected_Tag_Absolute'] / data['Selected_Tag_Absolute'].max()
    data['Selected_Tag_Relative_Normalized'] = data['Selected_Tag_Relative'] / data['Selected_Tag_Relative'].max()

    # 가중치를 적용하여 종합 점수 계산
    data['Score'] = (
        0.5 * data['Overall_Tag_Normalized'] +
        0.3 * data['Selected_Tag_Absolute_Normalized'] +
        0.2 * data['Selected_Tag_Relative_Normalized']
    )

    # 점수 기준 내림차순 정렬
    ranked_data = data.sort_values(by='Score', ascending=False)

    # 최소 7개 이상의 가게가 나오도록 조정
    if len(ranked_data) < 7:
        print(f"랭킹된 가게가 7개 미만입니다. 현재 {len(ranked_data)}개의 가게가 있습니다.")
    else:
        ranked_data = ranked_data.head(7)  # 상위 7개 가게 선택

    return ranked_data

# 사용자 입력 및 데이터 처리
try:
    # 데이터 선택
    selected_category = get_user_selection_multi(
        "데이터를 선택하세요 (1: 식당, 2: 카페):", ["식당", "카페"]
    )[0]  # 다중 선택 가능하지만 여기서는 첫 번째 선택만 사용

    if selected_category == "식당":
        selected_data = restaurant_data
        # 식당은 음식 종류 선택
        selected_food_category = get_user_selection_multi(
            "음식 종류를 선택하세요:", category_options["식당"]
        )
        print(f"선택된 음식 종류: {', '.join(selected_food_category)}")
        # 필터링: 선택된 음식 종류에 해당하는 데이터만 사용
        filtered_data = selected_data[selected_data['FoodCategory'].isin(selected_food_category)]
    else:
        selected_data = cafe_data
        filtered_data = selected_data  # 전체 데이터 사용
        selected_food_category = None  # 구분 없음

    # 태그 선택 (다중 선택)
    selected_tags = get_user_selection_multi(
        "중요하게 생각하는 태그를 선택하세요:", list(tag_options[selected_category].keys())
    )
    print(f"선택된 태그들: {', '.join(selected_tags)}")

except KeyError as e:
    print(f"선택지 처리 중 오류가 발생했습니다: {e}")
    raise

# 새로운 랭킹 함수 호출
ranked_results = rank_stores_custom_multi(filtered_data, selected_tags, tag_options[selected_category])

# 필터링된 결과가 없을 경우 모든 음식 종류에 해당하는 식당 반환
if ranked_results.empty:
    print(f"선택한 태그들 '{', '.join(selected_tags)}'에 부합하는 가게가 없습니다. 선택한 음식 종류에 해당하는 모든 가게를 반환합니다.")
    ranked_results = filtered_data

# 결과 출력
if not ranked_results.empty:
    print(f"랭킹된 가게들 (상위 {len(ranked_results)}개):")
    print(ranked_results[['가게명', 'FoodCategory', 'Score']])
else:
    print("선택한 조건에 해당하는 가게가 없습니다.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
데이터를 선택하세요 (1: 식당, 2: 카페):
1. 식당
2. 카페
선택 번호를 입력하세요 (쉼표로 구분): 1
음식 종류를 선택하세요:
1. 한식
2. 중식
3. 일식
4. 양식
5. 육류
6. 아시안음식
7. 샐러드
선택 번호를 입력하세요 (쉼표로 구분): 2,3
선택된 음식 종류: 중식, 일식
중요하게 생각하는 태그를 선택하세요:
1. 맛
2. 특별한 메뉴
3. 메뉴 다양성
4. 신선도
5. 가성비
6. 서비스
7. 청결
8. 혼밥
9. 다양한 술
10. 분위기
11. 대화하기 좋음
선택 번호를 입력하세요 (쉼표로 구분): 1,3,6
선택된 태그들: 맛, 메뉴 다양성, 서비스
랭킹된 가게들 (상위 7개):
          가게명 FoodCategory     Score
74    온센 용산구점           일식  0.976048
14          정           중식  0.475884
68        멘타미           일식  0.463569
2   면식당 숙명여대점           일식  0.384437
12      네코노스시           일식  0.380395
43     츄르츄르라멘           일식  0.335640
52     스시 사이꼬           일식  0.284810


<ipython-input-5-8116831839ef>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Selected_Tag_Absolute'] = data[selected_phrases].sum(axis=1)
<ipython-input-5-8116831839ef>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Overall_Tag_Count'] = data['태그 총합']
<ipython-input-5-8116831839ef>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [6]:
# 상위 태그 컬럼 제외
if selected_category == "식당":
  additional_excluded_columns = [
      "음식이 맛있어요",
      '빵이 맛있어요'
      '특별한 메뉴가 있어요',
      '메뉴 구성이 알차요',
      '재료가 신선해요',
      '잡내가 적어요',
      '고기 질이 좋아요'
      "친절해요",
      "가성비가 좋아요",
      "양이 많아요",
      '반찬이 잘 나와요',
      '음식이 빨리 나와요'
      "매장이 청결해요",
      '화장실이 깨끗해요'
  ]
else :
  additional_excluded_columns = [
      '커피가 맛있어요',
      '디저트가 맛있어요',
      '매장이 청결해요',
      '화장실이 깨끗해요',
      '친절해요',
      '포장이 깔끔해요',
      '아늑해요',
      '인테리어가 멋져요'
    ]

df= ranked_results

  # 첫 두 개 컬럼과 마지막 컬럼 제외
excluded_columns = df.columns[:3].tolist() + df.columns[-2:].tolist()

# 최종 제외 리스트
final_excluded_columns = excluded_columns + additional_excluded_columns

# 나머지 컬럼 리스트
remaining_columns = [col for col in df.columns if col not in final_excluded_columns]
print("Remaining Columns:", remaining_columns)

Remaining Columns: ['혼밥하기 좋아요', '친절해요', '특별한 메뉴가 있어요', '매장이 넓어요', '매장이 청결해요', '인테리어가 멋져요', '특별한 날 가기 좋아요', '단체모임 하기 좋아요', '아늑해요', '음식이 빨리 나와요', '주차하기 편해요', '뷰가 좋아요', '음료가 맛있어요', '아이와 가기 좋아요', '반려동물과 가기 좋아요', '룸이 잘 되어있어요', '오래 머무르기 좋아요', '환기가 잘 돼요', '차분한 분위기에요', '코스요리가 알차요', '비싼 만큼 가치있어요', '대화하기 좋아요', '건강한 맛이에요', '음악이 좋아요', '좌석이 편해요', '사진이 잘 나와요', '현지 맛에 가까워요', '고기 질이 좋아요', '샐러드바가 잘 되어있어요', '혼술하기 좋아요', '향신료가 강하지 않아요', '야외공간이 멋져요', '컨셉이 독특해요', '포장이 깔끔해요', '술이 다양해요', '직접 잘 구워줘요', '커피가 맛있어요', '디저트가 맛있어요', '기본 안주가 좋아요', '집중하기 좋아요', '빵이 맛있어요', '태그 총합', 'FoodCategory', 'Selected_Tag_Absolute', 'Overall_Tag_Count', 'Selected_Tag_Relative', 'Overall_Tag_Normalized', 'Selected_Tag_Absolute_Normalized']


In [7]:
# Input 텍스트와 Remaining Columns 정의
input_txt = [input("기타 입력 사항을 입력하시오: ")]

기타 입력 사항을 입력하시오: 혼밥


In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# KoBERT 모델 및 토크나이저 로드
model_name = "monologg/kobert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [9]:
# 문장 임베딩 생성 함수
def get_embeddings(text_list):
    inputs = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # 평균 풀링 사용
    return embeddings

# Threshold 설정
threshold = 0.72

# Input 텍스트와 Remaining Columns 임베딩
input_embeddings = get_embeddings(input_txt)
remaining_embeddings = get_embeddings(remaining_columns)

# 유사도 계산
similarity_matrix = cosine_similarity(input_embeddings.numpy(), remaining_embeddings.numpy())

# 유사도 결과를 데이터프레임으로 변환
similarity_df2 = pd.DataFrame(
    similarity_matrix,
    index=input_txt,
    columns=remaining_columns
)

# high_similarity_pairs 구성 및 결과 저장
high_similarity_pairs = []
result_data = []

for input_text in similarity_df2.index:
    similar_columns = similarity_df2.loc[input_text][similarity_df2.loc[input_text] >= threshold]

    # 상위 5개 선택 (유사도 기준 내림차순)
    selected_tags = similar_columns.sort_values(ascending=False).head(5)

    # 선택된 태그가 있다면 리스트로 저장, 없다면 빈 리스트
    if len(selected_tags) > 0:
        tag_list = selected_tags.index.tolist()
    else:
        tag_list = []

    # high_similarity_pairs에 추가
    for column, similarity in selected_tags.items():
        high_similarity_pairs.append((input_text, column, similarity))

    # 결과 데이터 구성
    result_data.append({"입력 텍스트": input_text, "최종 선택 태그의 리스트": tag_list})

# 결과 데이터프레임 생성
result_df = pd.DataFrame(result_data)

# 출력
print(result_df)

  입력 텍스트 최종 선택 태그의 리스트
0     혼밥            []


In [10]:
# 디버깅: result_df의 컬럼 확인
print("result_df 컬럼들:", result_df.columns)

# 각 input_txt에 대해 조건을 만족하는 가게별 유사도 점수 계산
final_results = []

for input_text in result_df["입력 텍스트"]:
    # 디버깅: 입력 텍스트 확인
    print(f"현재 처리 중: {input_text}")

    # 해당 input_txt의 tag_list 가져오기
    if "최종 선택 태그의 리스트" not in result_df.columns:
        raise KeyError("'최종 선택 태그의 리스트' 컬럼이 존재하지 않습니다. result_df 컬럼들을 확인하세요.")

    tag_list = result_df[result_df["입력 텍스트"] == input_text]["최종 선택 태그의 리스트"].values[0]

    # tag_list가 비어 있는 경우 처리
    if len(tag_list) == 0:
        final_results.append({
            "입력 텍스트": input_text,
            "추천 가게": [],
        })
        continue

    # 태그 수의 비율 계산 (1 / len(tag_list) * 태그 수)
    df["태그 비율"] = df[tag_list].sum(axis=1) / len(tag_list)

    # 조건에 맞는 가게 필터링 (태그 수 >= len(tag_list))
    filtered_df = df[df[tag_list].gt(0).sum(axis=1) >= len(tag_list)]

    # 태그 비율 기준 내림차순 정렬 후 상위 10개 가게 추출
    top_stores = filtered_df.sort_values("태그 비율", ascending=False).head(10)

    # 결과 저장
    recommended_stores = [
        {"가게명": row["가게명"], "유사도 점수": row["태그 비율"]}
        for _, row in top_stores.iterrows()
    ]

    final_results.append({
        "입력 텍스트": input_text,
        "추천 가게": recommended_stores,
    })

# 최종 결과 출력
for result2 in final_results:
    print(f"입력 텍스트: {result2['입력 텍스트']}")
    print("추천 가게:")
    for store in result2["추천 가게"]:
        print(f" - 가게명: {store['가게명']}, 유사도 점수: {store['유사도 점수']:.4f}")
    print()


result_df 컬럼들: Index(['입력 텍스트', '최종 선택 태그의 리스트'], dtype='object')
현재 처리 중: 혼밥
입력 텍스트: 혼밥
추천 가게:



# TF - IDF 리뷰 유사도 추출

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

input_txt =['초밥']

if selected_category == "식당":
    df_t= pd.read_csv('/content/drive/MyDrive/content/result_meal.csv')
else:
    df_t= pd.read_csv('/content/drive/MyDrive/content/result_cafe.csv')

# 가게명이 포함된 리스트가 ranked_results['가게명']에 있다고 가정
ranked_store_names = ranked_results['가게명']

# df_t에서 store_name이 ranked_store_names에 존재하는 경우만 필터링
df_t = df_t[df_t['store_name'].isin(ranked_store_names)]

# NaN 값 제거 또는 대체
df_t["text_review"] = df_t["text_review"].fillna("")  # NaN 값을 빈 문자열로 대체

# TF-IDF 벡터라이저 초기화
tfidf_vectorizer = TfidfVectorizer()

# 데이터셋 리뷰 벡터화
corpus = df_t["text_review"].tolist()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# 결과 저장을 위한 리스트
result_data = []

# 각 입력 텍스트에 대해 유사도 계산
for user_input in input_txt:
    # 입력 텍스트 벡터화
    input_vector = tfidf_vectorizer.transform([user_input])

    # 코사인 유사도 계산
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix)

    # 유사도 추가
    df_t["Similarity"] = similarity_scores.flatten()

    # 장소별 유사도 상위 5개의 점수 평균 계산
    top5_avg_similarity = (
        df_t.groupby("store_name")
        .apply(lambda group: group.nlargest(10, "Similarity")["Similarity"].mean())
        .reset_index(name="Avg_Similarity")
    )

    # 평균 유사도를 기준으로 상위 5개의 장소
    top5_places = top5_avg_similarity.sort_values(by="Avg_Similarity", ascending=False).head(10)

    # 결과 저장
    result_data.append({"입력 텍스트": user_input, "상위 5개 장소 및 평균 유사도": top5_places.to_dict(orient="records")})

# 결과 데이터프레임 생성
result_df_2 = pd.DataFrame(result_data)

# 출력
print(result_df_2)

  입력 텍스트                                  상위 5개 장소 및 평균 유사도
0     초밥  [{'store_name': '스시 사이꼬', 'Avg_Similarity': 0....


<ipython-input-11-cac3d60b3171>:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.nlargest(10, "Similarity")["Similarity"].mean())


In [12]:
# 유사도 점수가 높은 10개의 가게 선정 및 차등 가중치 적용
final_results = []

# 각 입력 텍스트에 대해 상위 10개 가게 선정 및 점수 계산
for result in result_data:
    user_input = result["입력 텍스트"]
    top_places = pd.DataFrame(result["상위 5개 장소 및 평균 유사도"])

    # 동일한 가게가 여러 입력 텍스트에 등장할 수 있으므로 모든 결과 합치기
    if not top_places.empty:
        # 상위 10개의 가게 및 평균 유사도 추출
        sorted_places = top_places.sort_values("Avg_Similarity", ascending=False).head(10)

        # 차등 가중치 계산: 10~5에서 0.5씩 감소
        weights = [200 - i * 10 for i in range(len(sorted_places))]

        # 최종 점수 계산: 유사도 × 가중치
        sorted_places["Weighted_Score"] = sorted_places["Avg_Similarity"] * weights

        # 최종 점수 기준으로 정렬
        sorted_places = sorted_places.sort_values("Weighted_Score", ascending=False)

        # 결과 저장
        final_results.append({
            "입력 텍스트": user_input,
            "추천 가게": sorted_places[["store_name", "Weighted_Score"]].to_dict(orient="records")
        })

# 최종 결과 출력
for result in final_results:
    print(f"입력 텍스트: {result['입력 텍스트']}")
    print("추천 가게:")
    for store in result["추천 가게"]:
        print(f" - 가게명: {store['store_name']}, 점수: {store['Weighted_Score']:.2f}")
    print()

입력 텍스트: 초밥
추천 가게:
 - 가게명: 스시 사이꼬, 점수: 18.00
 - 가게명: 멘타미, 점수: 0.00
 - 가게명: 면식당 숙명여대점, 점수: 0.00
 - 가게명: 온센 용산구점, 점수: 0.00
 - 가게명: 정, 점수: 0.00
 - 가게명: 츄르츄르라멘, 점수: 0.00



In [13]:
# 입력 텍스트별 최종 유사도 점수 계산
final_results_by_input = {}

# result와 result2의 점수 합산
for result, result2 in zip(final_results, final_results):
    input_text = result["입력 텍스트"]

    # 입력 텍스트에 대한 각 가게의 점수 합산
    store_scores = {}

    # result에서 점수 합산
    for store in result["추천 가게"]:
        store_name = store.get("store_name", store.get("가게명"))
        score = store.get("유사도 점수", store.get("Weighted_Score"))
        if store_name and score:
            if store_name not in store_scores:
                store_scores[store_name] = 0
            store_scores[store_name] += score

    # result2에서 점수 합산
    for store in result2["추천 가게"]:
        store_name = store.get("store_name", store.get("가게명"))
        score = store.get("유사도 점수", store.get("Weighted_Score"))
        if store_name and score:
            if store_name not in store_scores:
                store_scores[store_name] = 0
            store_scores[store_name] += score

    # 입력 텍스트별로 상위 5곳 추출
    sorted_stores = sorted(store_scores.items(), key=lambda x: x[1], reverse=True)[:5]
    final_results_by_input[input_text] = sorted_stores

# 결과 출력
for input_text, top_stores in final_results_by_input.items():
    print(f"입력 텍스트: {input_text}")
    print("최종 유사도 점수 상위 5곳:")
    for store_name, score in top_stores:
        print(f" - 가게명: {store_name}, 최종 점수: {score:.4f}")
    print()

입력 텍스트: 초밥
최종 유사도 점수 상위 5곳:
 - 가게명: 스시 사이꼬, 최종 점수: 36.0084



In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer


In [16]:
global raw_restaurant_dt
# 1. 데이터 로드 및 전처리
def load_and_preprocess(file_path_user, file_paths_restaurant):

    user_data = pd.read_csv(file_path_user)
    restaurant_data = pd.read_csv(file_paths_restaurant)

    # 사용자와 식당의 ID 추출
    user_ids = user_data['nickname'].tolist()
    restaurant_ids = restaurant_data['가게명'].tolist()

    # 숫자형 데이터 추출
    user_data_numeric = user_data.select_dtypes(include=['float64', 'int64'])
    restaurant_data_numeric = restaurant_data.select_dtypes(include=['float64', 'int64'])

    # 결측치 처리
    imputer = SimpleImputer(strategy='mean')
    user_data_imputed = imputer.fit_transform(user_data_numeric)
    restaurant_data_imputed = imputer.fit_transform(restaurant_data_numeric)

    # 비율 계산 (빈도수를 전체 합으로 나눔)
    restaurant_data_proportions = restaurant_data_imputed / restaurant_data_imputed.sum(axis=1, keepdims=True)

    # 정규화
    scaler = StandardScaler()
    user_data_scaled = scaler.fit_transform(user_data_imputed)
    restaurant_data_scaled = scaler.fit_transform(restaurant_data_proportions)

    return user_data_scaled, restaurant_data_scaled, user_ids, restaurant_ids, restaurant_data

# 2. PCA 수행
def perform_pca(data, n_components=3):

    pca = PCA(n_components=n_components)
    return pca.fit_transform(data)

# 3. 유사도 계산
def calculate_recommendation_scores(user_features, restaurant_features, weights=None):

    if weights is not None:
        # 가중치 적용
        weights = np.array(weights)
        restaurant_features = restaurant_features * weights
    return cosine_similarity(user_features, restaurant_features)


# 4. 추천 리스트 생성
def generate_recommendations(scores, user_ids, restaurant_ids, top_n=5):

    recommendations = []
    for i, user_id in enumerate(user_ids):
        top_indices = scores[i].argsort()[::-1][:top_n]
        top_restaurants = [restaurant_ids[j] for j in top_indices]
        top_scores = [scores[i][j] for j in top_indices]
        recommendations.append({"user_id": user_id, "recommended_restaurants": top_restaurants, "scores": top_scores})
    return pd.DataFrame(recommendations)

# 5. 결과 출력 함수
def print_recommendations(recommendations):

    for _, row in recommendations.iterrows():
        print(f"User ID: {row['user_id']}")
        print("Recommended Restaurants and Scores:")
        for restaurant, score in zip(row['recommended_restaurants'], row['scores']):
            print(f"  - {restaurant}: {score:.4f}")
        print()

def main(file_path_user, file_paths_restaurant, n_components=3, top_n=5):

    # 데이터 로드 및 전처리
    user_data, restaurant_data, user_ids, restaurant_ids, raw_restaurant_data = load_and_preprocess(
        file_path_user, file_paths_restaurant
    )

    # PCA 수행
    user_features = perform_pca(user_data, n_components)
    restaurant_features = perform_pca(restaurant_data, n_components)

    # 유사도 계산
    scores = calculate_recommendation_scores(user_features, restaurant_features)

    # 추천 결과 생성 및 출력
    recommendations = generate_recommendations(scores, user_ids, restaurant_ids, top_n)
    print_recommendations(recommendations)


# 실행
user_file_path="/content/drive/MyDrive/content/dacos_tagging_adjusted_sorted.csv"
restaurant_file_paths="/content/drive/MyDrive/content/combined_restaurant_data.csv"
main(user_file_path, restaurant_file_paths, n_components=3, top_n=5)

User ID: 369369z
Recommended Restaurants and Scores:
  - MUJABEE: 0.9858
  - Jesus Coffee: 0.9634
  - 때가이르매: 0.9497
  - 미드스트오브플로우: 0.9413
  - 와플하우스: 0.9389

User ID: A8뜨뚜
Recommended Restaurants and Scores:
  - 카페 퓨엔테: 0.9918
  - 와플하우스: 0.9861
  - 위드달걀빵: 0.9730
  - 스택빈 숙대점: 0.9532
  - 스타벅스 숙대점: 0.9420

User ID: Chiabata
Recommended Restaurants and Scores:
  - 미스터카츠 숙대입구점: 0.9989
  - 샤브로21 숙명여대: 0.9804
  - 나폴리키친: 0.9655
  - 네코노스시: 0.9603
  - 베스트프렌드: 0.9465

User ID: DEJAVU
Recommended Restaurants and Scores:
  - 카페나리나무: 0.9940
  - 컴포즈커피 숙명여대점: 0.9626
  - 루다브레드: 0.9092
  - 이웃집영준이: 0.8776
  - 평화남영: 0.8761

User ID: FromA
Recommended Restaurants and Scores:
  - 키보 에다마메: 0.9750
  - 벤티프레소 숙명여대점: 0.9706
  - 루다브레드: 0.9630
  - 없음 베이커리: 0.9466
  - 꽃보다크레페: 0.9435

User ID: Junns
Recommended Restaurants and Scores:
  - 마시앤바시: 0.8797
  - 사이공마켓 숙대점: 0.7937
  - 리또리또 숙대점: 0.6793
  - 포36거리 숙대점: 0.6096
  - 아리랑노점 숙대점: 0.6048

User ID: Reenfldos
Recommended Restaurants and Scores:
  - 스택빈 숙대점: 0.9144
  - 

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['index' '가게명']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
